# Import Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# Import the datasets

In [25]:
peopleFilepath = './Data/pickledPeople.pkl'
voteFilepath = './Data/pickledVotes_unpacked.pkl'
routeFilepath = './Data/pickledRoutes_unpacked.pkl'

people = pd.read_pickle(peopleFilepath)
votes = pd.read_pickle(voteFilepath)
routes = pd.read_pickle(routeFilepath)

## Function to find common people for an area

In [10]:
# Function to find all common people between N most-popular routes
def findCommonPeople(areaUrl, breadcrumbTier, nRoutes=10, routes=routes, people=people):
    areaRoutes = routes.loc[routes[breadcrumbTier]==areaUrl]
    areaRoutes = areaRoutes.sort_values('numQualityVotes', ascending=False)
    areaRoutes = areaRoutes[:nRoutes]
    
    subPeople = set([])
    
    # Filter the people dataset to only include those with at least nRoutes starRatings
    people = people.loc[people['numStarRatings']>=nRoutes*5]
            
    for i in range(len(areaRoutes)):
        tempSet=[]
        print("Parsing peopleDF for route", i+1, "of", nRoutes)
        for j in range(len(people)):
            if areaRoutes.index[i] in people.iloc[j]['starRatings']:
                tempSet.append(people.index[j])
                print(j/len(people), end='\r')        
        if i==0:
            subPeople=set(tempSet)
        else:
            subPeople = subPeople.intersection(set(tempSet))
            
    print(len(subPeople),"have climbed the", nRoutes, "most-rated climbs at", areaUrl) 

    return subPeople

## Isolate the People to be used for classification

In [54]:
eldoUrl = 'https://www.mountainproject.com/area/105744246/eldorado-canyon-sp'
subpeople = findCommonPeople(eldoUrl, 'bc3')


Parsing peopleDF for route 1 of 10


Parsing peopleDF for route 2 of 10
Parsing peopleDF for route 3 of 10
Parsing peopleDF for route 4 of 10
Parsing peopleDF for route 5 of 10
Parsing peopleDF for route 6 of 10
Parsing peopleDF for route 7 of 10
Parsing peopleDF for route 8 of 10
Parsing peopleDF for route 9 of 10
Parsing peopleDF for route 10 of 10
95 have climbed the 10 most-rated climbs at https://www.mountainproject.com/area/105744246/eldorado-canyon-sp


## Isolate the Routes to be used for classification

In [55]:
eldo = routes.loc[routes['bc3']=='https://www.mountainproject.com/area/105744246/eldorado-canyon-sp']
eldo = eldo.sort_values(by='numQualityVotes', ascending=False)
eldo = eldo[:10]

## Separate the Training and Testing dataset

In [138]:
# Make the training dataset
train = people.loc[subpeople[0]]
for i in range(1, int(len(subpeople)/2)):
    train = pd.concat([train, people.loc[subpeople[i]]], axis=1)
train = train.transpose()

# Make the test dataset
test = people.loc[subpeople[int(len(subpeople)/2)]]
for i in range(int(len(subpeople)/2)+1, len(subpeople)):
    test = pd.concat([test, people.loc[subpeople[i]]], axis=1)
test = test.transpose()

# All together, just in case it is needed
total = pd.concat([test, train])

### Visualization to see general star distribution

In [140]:
# Visualization to see general star distribution
d = {0:0, 1:0, 2:0, 3:0, 4:0}
r = list(eldo.index)
for i in range(len(total)):
    for j in range(len(eldo)):
        print(total.iloc[i]['starRatings'][r[j]], end=' ')
        d[total.iloc[i]['starRatings'][r[j]]] = d[total.iloc[i]['starRatings'][r[j]]]+1
    print()
    
for i in d:
    print(i, ":", d[i])

4 4 3 4 2 4 4 3 4 4 
4 4 4 3 4 3 4 4 3 4 
4 3 4 3 3 4 3 3 3 3 
3 4 4 3 3 3 4 4 2 4 
4 4 4 4 3 3 3 4 4 4 
3 3 4 2 2 3 2 2 2 3 
4 4 4 3 3 4 4 4 4 2 
3 3 4 4 2 3 3 3 3 4 
2 3 4 4 3 4 4 3 3 4 
3 3 2 3 2 3 3 2 2 3 
4 3 3 4 3 4 4 4 3 3 
4 3 4 4 3 3 4 4 4 4 
4 3 4 4 2 3 4 3 3 3 
3 4 3 3 3 3 4 4 3 4 
4 4 4 4 4 4 4 4 4 4 
3 3 4 3 3 3 4 4 3 3 
2 4 4 3 3 3 4 3 4 2 
4 4 4 4 3 3 4 4 3 3 
4 4 4 3 3 4 4 4 3 3 
3 4 4 2 3 4 4 4 4 4 
2 4 4 3 3 2 4 4 4 3 
4 4 4 4 3 3 4 4 3 4 
4 3 4 4 3 3 4 3 3 2 
3 3 4 2 3 3 4 4 3 4 
4 3 4 3 3 3 4 4 3 3 
3 3 4 2 3 3 4 3 3 3 
3 4 4 3 3 4 4 4 4 4 
4 4 4 3 3 4 4 4 4 4 
4 4 2 3 3 4 4 3 3 4 
4 4 4 4 3 3 4 4 4 4 
4 3 4 4 4 4 4 4 4 4 
4 4 4 3 3 3 3 3 3 3 
4 4 4 3 3 3 4 3 4 3 
3 3 4 3 3 3 3 3 3 3 
3 4 4 3 3 4 4 4 4 4 
4 4 4 2 2 3 4 3 2 3 
4 4 4 3 2 4 4 4 4 4 
3 4 3 2 3 3 4 4 2 3 
4 4 4 4 4 3 4 4 4 4 
4 4 4 3 4 4 4 3 4 4 
4 3 4 2 3 3 4 2 2 3 
4 4 4 3 3 4 4 4 4 4 
4 3 2 4 3 3 3 4 4 3 
2 3 3 2 3 3 4 3 4 4 
4 4 4 3 4 3 4 4 2 4 
4 3 4 3 2 3 4 3 4 3 
4 4 4 3 3 4 3 4 3 4 
4 3 4 4 3 4 4

In [133]:
test.iloc[0]['starRatings']['https://www.mountainproject.com/route/105748490/the-bastille-crack']

4